In [2]:
!pip install folium
import folium
import pandas as pd
import numpy as np

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
# define the world map
world_map = folium.Map()

# display world map
world_map

In [6]:
def fetch_data_and_save_to_df():
    try:
        csv_url = 'https://api.example.com/data'

        # Send an HTTP GET request to the CSV download link
        response = requests.get(csv_url)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        # Save the CSV content to a local file
        with open('data.csv', 'wb') as csv_file:
            csv_file.write(response.content)
        
        # Convert the fetched data into a pandas DataFrame
        spacex_df = pd.read_csv('data.csv')
        
        # Process and manipulate the data in the DataFrame as needed
        # For example, you can filter, clean, or analyze the data here
        
        # Print the DataFrame
        print(spacex_df)
        
        # Optionally, you can save the DataFrame to a CSV file
        #spacex_df.to_csv('data.csv', index=False)  # Change the filename as needed

    except requests.exceptions.RequestException as e:
        print('Request error:', e)
    except Exception as e:
        print('Error:', e)

fetch_data_and_save_to_df()

NameError: name 'requests' is not defined

In [17]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io
!pip install PyExecJS
import request

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = request.get(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

ImportError: cannot import name 'fetch' from 'js' (C:\Users\Vibhav\anaconda3\lib\site-packages\js\__init__.py)

In [7]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

# Use pandas to read the CSV file and store it in a DataFrame
spacex_df = pd.read_csv(url)

In [8]:
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [9]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df.dtypes

Launch Site     object
Lat            float64
Long           float64
dtype: object

In [10]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [12]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for site, lat, long in zip(launch_sites_df["Launch Site"], launch_sites_df["Lat"], launch_sites_df["Long"]):
    site_name = site
    site_latitude = lat
    site_longitude = long

    # Create a CircleMarker for the launch site
    circle_marker = folium.CircleMarker(
        location=[site_latitude, site_longitude],
        radius=10,  # Adjust the radius as needed
        color='blue',  # Color of the circle
        fill=True,
        fill_color='blue',  # Fill color of the circle
        fill_opacity=0.6,
        popup=site_name  # Popup label with the launch site name
    )

    # Add the CircleMarker to the map
    circle_marker.add_to(site_map)

In [13]:
site_map

In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
color_mapping = {1: 'blue', 0 : 'red'}
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for site, lat, long, color in zip(spacex_df["Launch Site"], spacex_df["Lat"], spacex_df["Long"], spacex_df["class"]):
    site_name = site
    site_latitude = lat
    site_longitude = long
    
    colors = color_mapping.get(color, 'gray')
    # Create a CircleMarker for the launch site
    circle_marker = folium.CircleMarker(
        location=[site_latitude, site_longitude],
        radius=10,  # Adjust the radius as needed
        color=colors,  # Color of the circle
        fill=True,
        fill_color= colors,  # Fill color of the circle
        fill_opacity=0.6,
        popup=site_name  # Popup label with the launch site name
    )

    # Add the CircleMarker to the map
    circle_marker.add_to(site_map)

In [15]:
site_map

In [16]:
marker_cluster = MarkerCluster()

In [17]:
# Define a function to apply to column 'A' and create a new column 'Color'
def assign_color(value):
    if value == 1:
        return 'green'
    else:
        return 'red'

# Use the apply method to create a new column 'Color' based on the condition
spacex_df['Color'] = spacex_df['class'].apply(assign_color)


In [18]:
spacex_df.head

<bound method NDFrame.head of      Launch Site        Lat        Long  class  Color
0    CCAFS LC-40  28.562302  -80.577356      0    red
1    CCAFS LC-40  28.562302  -80.577356      0    red
2    CCAFS LC-40  28.562302  -80.577356      0    red
3    CCAFS LC-40  28.562302  -80.577356      0    red
4    CCAFS LC-40  28.562302  -80.577356      0    red
5    CCAFS LC-40  28.562302  -80.577356      0    red
6    CCAFS LC-40  28.562302  -80.577356      0    red
7    CCAFS LC-40  28.562302  -80.577356      0    red
8    CCAFS LC-40  28.562302  -80.577356      0    red
9    CCAFS LC-40  28.562302  -80.577356      0    red
10   CCAFS LC-40  28.562302  -80.577356      0    red
11   CCAFS LC-40  28.562302  -80.577356      0    red
12   CCAFS LC-40  28.562302  -80.577356      0    red
13   CCAFS LC-40  28.562302  -80.577356      0    red
14   CCAFS LC-40  28.562302  -80.577356      0    red
15   CCAFS LC-40  28.562302  -80.577356      0    red
16   CCAFS LC-40  28.562302  -80.577356      0    re

In [19]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Use the coordinates from the DataFrame
        popup=record['Launch Site'],  # Customize the popup text if needed
        icon=folium.Icon(color='white', icon_color=record['Color'])  # Customize the icon color
    )
    marker_cluster.add_child(marker)

site_map

In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [23]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
distance_coastline = []
for lat, long in zip(spacex_df['Lat'], spacex_df['Long']):
    distance_coastline = calculate_distance(lat, long, 28.56367, -80.57163)

In [24]:
distance_coastline

0.5097439631188213

In [26]:
 distance_marker = folium.Marker(
    location = [28.56367, -80.57163],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

In [32]:
for lat, long in zip(spacex_df['Lat'], spacex_df['Long']):
    lines=folium.PolyLine(((lat,long),(28.56391, -80.56781)), weight=1)
    site_map.add_child(lines)

In [33]:
site_map

In [34]:
highway = [28.56336, -80.57081]
railway = [28.57234, -80.58528]
city = [28.39972, -80.60537]

In [35]:
 distance_marker = folium.Marker(
    location = highway,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

In [36]:
for lat, long in zip(spacex_df['Lat'], spacex_df['Long']):
    lines=folium.PolyLine(((lat,long),highway), weight=1)
    site_map.add_child(lines)

In [37]:
site_map